torchtext는 자연어 처리 분야에서 사용하는 데이터로더이다.파일 가져오기, 토큰화, 단어 집합 생성, 인코딩, 단어 벡터 생성 등의 작업을 지원하기 때문에 자연어 처리에서 많이 사용되고 있다.

In [5]:
import torch
torch.__version__

'1.11.0+cu113'

In [7]:
pip install torchtext==0.9

  Using cached torch-1.8.0-cp38-cp38-manylinux1_x86_64.whl (735.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
pytorch-lightning 1.7.6 requires torch>=1.9.*, but you have torch 1.8.0 which is incompatible.
You should consider upgrading via the '/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torchtext # 0.9
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

## 데이터 전처리

In [2]:
start = time.time()
TEXT = torchtext.legacy.data.Field(lower=True, fix_length=200, batch_first=False)
LABEL = torchtext.legacy.data.Field(sequential=False)

* torchtext.legacy.data에서 제공하는 Field는 데이터 전처리를 위해 사용되며 여기에서 사용되는 파라미터는 다음과 같다.
    1. lower: 대문자를 모두 소문자로 변경한다. 기본값은 false
    2. fix_length: 고정된 길이의 데이터를 얻을 수 있다. 여기에서는 데이터의 길이를 200으로 고정했으며 200보다 짧다면 패딩을 시켜준다.
    3. batch_first: 신경망에 입력되는 텐서의 첫 번째 차원 값이 배치 크기가 되도록 한다.

## 데이터셋 준비

In [3]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [4]:
print(vars(train_data.examples[0])) # 데이터셋의 내용을 보고자 할 때는 examples를 사용

{'text': ['for', 'the', 'life', 'of', 'me,', 'why', 'did', 'this', 'film', 'receive', 'an', 'r', 'rating?!', 'while', 'it', 'is', 'about', 'flesh-eating', 'zombies,', 'believe', 'it', 'or', 'not,', "it's", 'actually', 'a', 'pretty', 'good', 'family-friendly', 'film--at', 'least', 'if', 'your', 'kids', 'are', 'age', '10', 'and', 'older.', 'unlike', 'the', 'traditional', 'zombie', 'films,', 'this', 'one', 'has', 'an', 'excellent', 'sense', 'of', 'humor', 'as', 'well', 'as', 'a', 'traditional', 'values--albeit', 'a', 'bit', 'twisted!', 'the', 'language', "isn't", 'a', 'serious', 'problem,', 'there', 'is', 'no', 'nudity', 'and', 'the', 'film', 'style', 'is', 'definitely', 'geared', 'towards', 'kids', '(much', 'like', 'the', 'old', 'tv', 'show', '"eerie,', 'indiana")--yet', 'some', 'knucklehead', 'slapped', 'an', 'r', 'rating', 'one', 'it!', 'believe', 'me,', 'most', 'kids', 'have', 'seen', 'worse', 'violence', 'than', 'this', 'and', 'it', 'just', 'seems', 'silly', 'to', 'make', 'audiences'

## 데이터셋 전처리 적용

In [5]:
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)['text']] # 소문자로 변경
    text = [x.replace('<br', '') for x in text] # '<br'을 ''으로 변경
    text = [''.join(c for c in s if c not in string.punctuation) for s in text] # 구두점 제거
    text = [s for s in text if s] # 공백 제거
    vars(example)['text'] = text

## 훈련과 검증 데이터셋 분리

In [6]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [7]:
print('Number of training examples', len(train_data))
print('Number of validation examples', len(valid_data))
print('Number of testing examples', len(test_data))

Number of training examples 20000
Number of validation examples 5000
Number of testing examples 25000


## 단어 집합 만들기

In [8]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

In [9]:
print('Unique tokens is TEXT covab: ', len(TEXT.vocab))
print('Unique tokens is LABEL covab: ', len(LABEL.vocab))

Unique tokens is TEXT covab:  10002
Unique tokens is LABEL covab:  3


* max_size : 단어 집합의 크기로 단어 집합에 포함되는 어휘 수를 의미한다.
* min_freq : 훈련 데이터셋에서 특정 단어의 최소 등장 횟수를 의미한다.
* vectors : 임베딩 벡터를 지정할 수 있다. 임베딩 벡터는 워드 임베딩의 결과로 나온 벡터이다. 

### 테스트 데이터셋의 단어 집합 확인

In [10]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fdae053b760>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


## 데이터셋 메모리로 가져오기

In [11]:
BATCH_SIZE = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embedding_dim = 100 # 각 단어를 100차원으로 조정
hidden_size = 300

train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
                                                (train_data, valid_data, test_data),
                                                batch_size=BATCH_SIZE,
                                                device=device)

* BucketIterator는 데이터로더와 쓰임새가 같다. 즉, 배치 크기 단위로 값을 차례대로 꺼내어 메모리로 가져오고 싶을 때 사용한다. 특히 Field에서 fix_length를 사용하지 않았다면 BucketIterator는 비슷한 길이의 데이터를 한 배치에 할당하여 패딩을 최소화시켜 준다.

## 워드 임베딩 및 RNN 셀 정의

In [12]:
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size) # ⓵
        
    def forward(self, inputs):
        bz = inputs.shape[1] # 배치
        ht = torch.zeros((bz, hidden_size)).to(device) # 배치와 은닉층 뉴런의 크기를 0으로 초기화
        for word in inputs:
            ht = self.rnn(word, ht) # ⓶
            
        return ht
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi), embedding_dim) # ⓷
        self.rnn = RNNCell_Encoder(embedding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)
        
    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

⓶ : 재귀적으로 발생하는 상태 값을 처리하기 위한 구문이다.  
* ht : 현재 상태를 의미하는 것  
* word : 현재의 입력 벡터로 (Batch, input_size)형태를 갖는다.  
* ht : 이전 상태를 의미하는 것


## 옵티마이저와 손실 함수 정의

In [13]:
model = Net()
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

## 모델 학습을 위한 함수 정의

In [14]:
def training(epoch, model, trainloader, validloader):
    correct = 0
    total = 0
    running_loss = 0
    
    model.train()
    for b in trainloader:
        x, y = b.text, b.label # trainloader에서 text와 label을 꺼내온다.
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
            
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
    
    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0
    
    model.eval()
    with torch.no_grad():
        for b in validloader:
            x, y = b.text, b.label
            x, y= x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            valid_correct += (y_pred == y).sum().item()
            valid_total += y.size(0)
            valid_running_loss += loss.item()
            
    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total
    
    print('epoch: ', epoch,
          'loss: ', round(epoch_loss, 3),
          'accuracy: ', round(epoch_acc, 3),
          'valid_loss: ', round(epoch_valid_loss, 3),
          'valid_acc: ', round(epoch_valid_acc, 3)
         )
    
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

## 모델 학습

In [15]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(
                                                                epoch, model, train_iterator, valid_iterator)
    train_loss.append(epoch_loss) # 훈련 데이텃세을 모델에 적용했을 때의 오차
    train_acc.append(epoch_acc) # 훈련 데이터셋을 모델에 적용했을 때의 정확도
    valid_loss.append(epoch_valid_loss) # 검증 데이터셋을 모델에 적용했을 때의 오차
    valid_acc.append(epoch_valid_acc) # 검증 데이터셋을 모델에 적용했을 때의 오차

RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasCreate(handle)`